![alt text](http://pandas.pydata.org/_static/pandas_logo.png)

<center><h1> PANDAS TABLE OF CONTENTS </h1></center>

## [Group By: split-apply-combine](http://pandas.pydata.org/pandas-docs/stable/groupby.html) 

- [Splitting-and-Replacing-Strings](#Splitting-and-Replacing-Strings)

In [1]:
import pandas as pd
import numpy as np
from pivottablejs import pivot_ui

By “group by” we are referring to a process involving one or more of the following steps

* **Splitting** the data into groups based on some criteria
* **Applying** a function to each group independently
* **Combining** the results into a data structure

Of these, the split step is the most straightforward. In fact, in many situations you may wish to split the data set into groups and do something with those groups yourself. In the apply step, we might wish to one of the following:

* **Aggregation:** computing a summary statistic (or statistics) about each group. Some examples:

  * Compute group sums or means
  * Compute group sizes / counts

* **Transformation:** perform some group-specific computations and return a like-indexed. Some examples:

  * Standardizing data (zscore) within group
  * Filling NAs within groups with a value derived from each group

* **Filtration:** discard some groups, according to a group-wise computation that evaluates True or False. Some examples:

  * Discarding data that belongs to groups with only a few members
  * Filtering out data based on the group sum or mean

* Some combination of the above: GroupBy will examine the results of the apply step and try to return a sensibly combined result if it doesn’t fit into either of the above two categories

Since the set of object instance method on pandas data structures are generally rich and expressive, we often simply want to invoke, say, a DataFrame function on each group. The name GroupBy should be quite familiar to those who have used a SQL-based tool (or itertools), in which you can write code like:

    SELECT Column1, Column2, mean(Column3), sum(Column4)
    FROM SomeTable
    GROUP BY Column1, Column2

We aim to make operations like this natural and easy to express using pandas. We’ll address each area of GroupBy functionality then provide some non-trivial examples / use cases.

See the [cookbook](http://pandas.pydata.org/pandas-docs/stable/cookbook.html#cookbook-grouping) for some advanced strategies

## Splitting an object into groups

pandas objects can be split on any of their axes. The abstract definition of grouping is to provide a mapping of labels to group names. To create a GroupBy object (more on what the GroupBy object is later), you do the following:

    >>> grouped = obj.groupby(key)
    >>> grouped = obj.groupby(key, axis=1)
    >>> grouped = obj.groupby([key1, key2])

The mapping can be specified many different ways:

* A Python function, to be called on each of the axis labels
* A list or NumPy array of the same length as the selected axis
* A dict or Series, providing a label -> group name mapping
* For DataFrame objects, a string indicating a column to be used to group. Of course df.groupby('A') is just syntactic sugar for df.groupby(df['A']), but it makes life simpler
* A list of any of the above things

Collectively we refer to the grouping objects as the **keys**. For example, consider the following DataFrame:

In [2]:
df = pd.DataFrame({'A' : ['foo', 'bar', 'foo', 'bar',
                           'foo', 'bar', 'foo', 'foo'],
                   'B' : ['one', 'one', 'two', 'three',
                          'two', 'two', 'one', 'three'],
                   'C' : np.random.randn(8),
                   'D' : np.random.randn(8)})

In [3]:
pivot_ui(df)

We could naturally group by either the A or B columns or both:

In [4]:
grouped = df.groupby('A')

In [5]:
grouped.count()

,B,C,D
A,,,
bar,3,3,3
foo,5,5,5


In [6]:
grouped = df.groupby(['A', 'B'])

In [7]:
grouped.count()

C  D
A   B          
bar one    1  1
    three  1  1
    two    1  1
foo one    2  2
    three  1  1
    two    2  2

In [8]:
def get_letter_type(letter):
    if letter.lower() in 'aeiou':
        return 'vowel'
    else:
        return 'consonant'

In [9]:
grouped = df.groupby(get_letter_type, axis=1)

In [10]:
grouped.count().iloc[0]

consonant    3
vowel        1
Name: 0, dtype: int64

Starting with 0.8, pandas Index objects now supports duplicate values. If a non-unique index is used as the group key in a groupby operation, all values for the same index value will be considered to be in one group and thus the output of aggregation functions will only contain unique index values:

In [11]:
lst = [1, 2, 3, 1, 2, 3]

In [12]:
s = pd.Series([1, 2, 3, 10, 20, 30], lst)

In [13]:
s

1     1
2     2
3     3
1    10
2    20
3    30
dtype: int64

In [14]:
grouped = s.groupby(level=0)

In [15]:
grouped.first()

1    1
2    2
3    3
dtype: int64

In [16]:
grouped.last()

1    10
2    20
3    30
dtype: int64

In [17]:
grouped.sum()

1    11
2    22
3    33
dtype: int64

Note that **no splitting occurs** until it’s needed. Creating the GroupBy object only verifies that you’ve passed a valid mapping.

**Note:** Many kinds of complicated data manipulations can be expressed in terms of GroupBy operations (though can’t be guaranteed to be the most efficient). You can get quite creative with the label mapping functions.

### GroupBy sorting

By default the group keys are sorted during the groupby operation. You may however pass sort=False for potential speedups:

In [18]:
df2 = pd.DataFrame({'X' : ['B', 'B', 'A', 'A'], 'Y' : [1, 2, 3, 4]})

In [19]:
df2

,X,Y
0,B,1
1,B,2
2,A,3
3,A,4


In [20]:
%%time
df2.groupby(['X']).sum()

Wall time: 2 ms


,Y
X,
A,7
B,3


In [21]:
%%time
df2.groupby(['X'], sort=False).sum()

Wall time: 2 ms


,Y
X,
B,3
A,7


Note that groupby will preserve the order in which observations are sorted within each group. For example, the groups created by groupby() below are in the order the appeared in the original DataFrame:

In [22]:
df3 = pd.DataFrame({'X' : ['A', 'B', 'A', 'B'], 'Y' : [1, 4, 3, 2]})

In [23]:
df3

,X,Y
0,A,1
1,B,4
2,A,3
3,B,2


In [24]:
df3.groupby(['X']).get_group('A')

,X,Y
0,A,1
2,A,3


In [25]:
df3.groupby(['X']).get_group('B')

,X,Y
1,B,4
3,B,2


### GroupBy object attributes

The groups attribute is a dict whose keys are the computed unique groups and corresponding values being the axis labels belonging to each group. In the above example we have:

In [26]:
df

,A,B,C,D
0,foo,one,0.364497,0.593361
1,bar,one,-1.281965,1.046477
2,foo,two,1.491031,-1.514367
3,bar,three,-0.611272,-0.629967
4,foo,two,0.744479,0.525437
5,bar,two,0.526694,-0.036847
6,foo,one,-0.506025,0.629236
7,foo,three,-0.475310,0.327859


In [27]:
df.groupby('A').groups

{'bar': [1L, 3L, 5L], 'foo': [0L, 2L, 4L, 6L, 7L]}

In [28]:
 df.groupby(get_letter_type, axis=1).groups

{'consonant': ['B', 'C', 'D'], 'vowel': ['A']}

Calling the standard Python len function on the GroupBy object just returns the length of the groups dict, so it is largely just a convenience:

In [29]:
grouped = df.groupby(['A', 'B'])

In [30]:
grouped.groups

{('bar', 'one'): [1L],
 ('bar', 'three'): [3L],
 ('bar', 'two'): [5L],
 ('foo', 'one'): [0L, 6L],
 ('foo', 'three'): [7L],
 ('foo', 'two'): [2L, 4L]}

In [31]:
len(grouped)

6

GroupBy will tab complete column names (and other attributes)

### GroupBy with MultiIndex

With [hierarchically-indexed](http://pandas.pydata.org/pandas-docs/stable/advanced.html#advanced-hierarchical) data, it’s quite natural to group by one of the levels of the hierarchy.

Let’s create a series with a two-level MultiIndex.

In [32]:
arrays = [['bar', 'bar', 'baz', 'baz', 'foo', 'foo', 'qux', 'qux'],
          ['one', 'two', 'one', 'two', 'one', 'two', 'one', 'two']]

In [33]:
index = pd.MultiIndex.from_arrays(arrays, names=['first', 'second'])

In [34]:
s = pd.Series(np.random.randn(8), index=index)

In [35]:
s

first  second
bar    one      -0.702736
       two      -0.390943
baz    one      -1.112570
       two       1.042282
foo    one       0.104071
       two       3.058415
qux    one       2.071738
       two      -0.667976
dtype: float64

We can then group by one of the levels in s.

In [36]:
grouped = s.groupby(level=0)

In [37]:
grouped.sum()

first
bar   -1.093680
baz   -0.070288
foo    3.162486
qux    1.403762
dtype: float64

If the MultiIndex has names specified, these can be passed instead of the level number:

In [38]:
s.groupby(level='second').sum()

second
one    0.360503
two    3.041778
dtype: float64

The aggregation functions such as sum will take the level parameter directly. Additionally, the resulting index will be named according to the chosen level:

In [39]:
s.sum(level='second')

second
one    0.360503
two    3.041778
dtype: float64

In [40]:
arrays = [['bar', 'bar', 'baz', 'baz', 'foo', 'foo', 'qux', 'qux'],
          ['doo', 'doo', 'bee', 'bee', 'bop', 'bop', 'bop', 'bop'],
          ['one', 'two', 'one', 'two', 'one', 'two', 'one', 'two']]

In [41]:
index = pd.MultiIndex.from_arrays(arrays, names=['first', 'second','third'])

In [42]:
s = pd.Series(np.random.randn(8), index=index)

In [43]:
s

first  second  third
bar    doo     one      1.096589
               two     -0.395676
baz    bee     one      0.272966
               two      1.288253
foo    bop     one      0.227814
               two     -0.037481
qux    bop     one     -0.260992
               two     -1.144675
dtype: float64

In [44]:
s.groupby(level=['first', 'second']).sum()

first  second
bar    doo       0.700913
baz    bee       1.561219
foo    bop       0.190333
qux    bop      -1.405667
dtype: float64

More on the sum function and aggregation later.

### DataFrame column selection in GroupBy

Once you have created the GroupBy object from a DataFrame, for example, you might want to do something different for each of the columns. Thus, using [] similar to getting a column from a DataFrame, you can do:

In [45]:
df

,A,B,C,D
0,foo,one,0.364497,0.593361
1,bar,one,-1.281965,1.046477
2,foo,two,1.491031,-1.514367
3,bar,three,-0.611272,-0.629967
4,foo,two,0.744479,0.525437
5,bar,two,0.526694,-0.036847
6,foo,one,-0.506025,0.629236
7,foo,three,-0.475310,0.327859


In [46]:
grouped = df.groupby(['A'])

In [47]:
grouped_C = grouped['C']

In [48]:
grouped_D = grouped['D']

This is mainly syntactic sugar for the alternative and much more verbose:

In [49]:
df['C'].groupby(df['A'])

Additionally this method avoids recomputing the internal grouping information derived from the passed key.

## Iterating through groups

With the GroupBy object in hand, iterating through the grouped data is very natural and functions similarly to itertools.groupby:

In [50]:
grouped = df.groupby('A')

In [51]:
for name, group in grouped:
       print name 
       print '-' * len(name)
       print group
       print '\n'

bar
---
     A      B         C         D
1  bar    one -1.281965  1.046477
3  bar  three -0.611272 -0.629967
5  bar    two  0.526694 -0.036847


foo
---
     A      B         C         D
0  foo    one  0.364497  0.593361
2  foo    two  1.491031 -1.514367
4  foo    two  0.744479  0.525437
6  foo    one -0.506025  0.629236
7  foo  three -0.475310  0.327859




In the case of grouping by multiple keys, the group name will be a tuple:

In [52]:
for name, group in df.groupby(['A', 'B']):
       print name 
       print '-' * 15
       print group
       print '\n'

('bar', 'one')
---------------
     A    B         C         D
1  bar  one -1.281965  1.046477


('bar', 'three')
---------------
     A      B         C         D
3  bar  three -0.611272 -0.629967


('bar', 'two')
---------------
     A    B         C         D
5  bar  two  0.526694 -0.036847


('foo', 'one')
---------------
     A    B         C         D
0  foo  one  0.364497  0.593361
6  foo  one -0.506025  0.629236


('foo', 'three')
---------------
     A      B        C         D
7  foo  three -0.47531  0.327859


('foo', 'two')
---------------
     A    B         C         D
2  foo  two  1.491031 -1.514367
4  foo  two  0.744479  0.525437




It’s standard Python-fu but remember you can unpack the tuple in the for loop statement if you wish: for (k1, k2), group in grouped:.

## Selecting a group

A single group can be selected using GroupBy.get_group():

In [53]:
grouped.get_group('bar')

,A,B,C,D
1,bar,one,-1.281965,1.046477
3,bar,three,-0.611272,-0.629967
5,bar,two,0.526694,-0.036847


Or for an object grouped on multiple columns:

In [54]:
df.groupby(['A', 'B']).get_group(('bar', 'one'))

,A,B,C,D
1,bar,one,-1.281965,1.046477


## Aggregation

Once the GroupBy object has been created, several methods are available to perform a computation on the grouped data.

An obvious one is aggregation via the aggregate or equivalently agg method:

In [55]:
grouped = df.groupby('A')

In [56]:
grouped.aggregate(np.sum)

,C,D
A,,
bar,-1.366543,0.379664
foo,1.618672,0.561526


In [57]:
grouped = df.groupby(['A', 'B'])

In [58]:
grouped.aggregate(np.sum)

C         D
A   B                        
bar one   -1.281965  1.046477
    three -0.611272 -0.629967
    two    0.526694 -0.036847
foo one   -0.141528  1.222597
    three -0.475310  0.327859
    two    2.235510 -0.988930

As you can see, the result of the aggregation will have the group names as the new index along the grouped axis. In the case of multiple keys, the result is a [MultiIndex](http://pandas.pydata.org/pandas-docs/stable/advanced.html#advanced-hierarchical) by default, though this can be changed by using the as_index option:

In [59]:
grouped = df.groupby(['A', 'B'], as_index=False)

In [60]:
grouped.aggregate(np.sum)

,A,B,C,D
0,bar,one,-1.281965,1.046477
1,bar,three,-0.611272,-0.629967
2,bar,two,0.526694,-0.036847
3,foo,one,-0.141528,1.222597
4,foo,three,-0.475310,0.327859
5,foo,two,2.235510,-0.988930


In [61]:
df.groupby('A', as_index=False).sum()

,A,C,D
0,bar,-1.366543,0.379664
1,foo,1.618672,0.561526


Note that you could use the reset_index DataFrame function to achieve the same result as the column names are stored in the resulting MultiIndex:

In [62]:
df.groupby(['A', 'B']).sum()

C         D
A   B                        
bar one   -1.281965  1.046477
    three -0.611272 -0.629967
    two    0.526694 -0.036847
foo one   -0.141528  1.222597
    three -0.475310  0.327859
    two    2.235510 -0.988930

In [63]:
df.groupby(['A', 'B']).sum().reset_index()

,A,B,C,D
0,bar,one,-1.281965,1.046477
1,bar,three,-0.611272,-0.629967
2,bar,two,0.526694,-0.036847
3,foo,one,-0.141528,1.222597
4,foo,three,-0.475310,0.327859
5,foo,two,2.235510,-0.988930


Another simple aggregation example is to compute the size of each group. This is included in GroupBy as the size method. It returns a Series whose index are the group names and whose values are the sizes of each group.

In [64]:
grouped = df.groupby(['A', 'B'])

In [65]:
grouped.size()

A    B    
bar  one      1
     three    1
     two      1
foo  one      2
     three    1
     two      2
dtype: int64

In [66]:

grouped['A'].count()

A    B    
bar  one      1
     three    1
     two      1
foo  one      2
     three    1
     two      2
Name: A, dtype: int64

In [67]:
grouped.describe().head(8)

C         D
A   B                            
bar one count  1.000000  1.000000
        mean  -1.281965  1.046477
        std         NaN       NaN
        min   -1.281965  1.046477
        25%   -1.281965  1.046477
        50%   -1.281965  1.046477
        75%   -1.281965  1.046477
        max   -1.281965  1.046477

**Note:** Aggregation functions **will not** return the groups that you are aggregating over if they are named columns, when as_index=True, the default. The grouped columns will be the **indices** of the returned object.

Passing as_index=False **will** return the groups that you are aggregating over, if they are named columns.

Aggregating functions are ones that reduce the dimension of the returned objects, for example: mean, sum, size, count, std, var, sem, describe, first, last, nth, min, max. This is what happens when you do for example DataFrame.sum() and get back a Series.

nth can act as a reducer or a filter, see [here](http://pandas.pydata.org/pandas-docs/stable/groupby.html#groupby-nth)

### Applying multiple functions at once

With grouped Series you can also pass a list or dict of functions to do aggregation with, outputting a DataFrame:

In [68]:
grouped = df.groupby('A')

In [69]:
grouped['C'].agg([np.sum, np.mean, np.std])

,sum,mean,std
A,,,
bar,-1.366543,-0.455514,0.914334
foo,1.618672,0.323734,0.846794


If a dict is passed, the keys will be used to name the columns. Otherwise the function’s name (stored in the function object) will be used.

In [70]:
grouped['D'].agg({'result1' : np.sum,
                  'result2' : np.mean})

,result2,result1
A,,
bar,0.126555,0.379664
foo,0.112305,0.561526


On a grouped DataFrame, you can pass a list of functions to apply to each column, which produces an aggregated result with a hierarchical index:

In [71]:
grouped.agg([np.sum, np.mean, np.std])

C                             D                    
          sum      mean       std       sum      mean       std
A                                                              
bar -1.366543 -0.455514  0.914334  0.379664  0.126555  0.850083
foo  1.618672  0.323734  0.846794  0.561526  0.112305  0.916766

Passing a dict of functions has different behavior by default, see the next section.

### Applying different functions to DataFrame columns

By passing a dict to aggregate you can apply a different aggregation to the columns of a DataFrame:

In [72]:
grouped.agg({'C' : np.sum,
             'D' : lambda x: np.std(x, ddof=1)})

,C,D
A,,
bar,-1.366543,0.850083
foo,1.618672,0.916766


The function names can also be strings. In order for a string to be valid it must be either implemented on GroupBy or available via [dispatching](http://pandas.pydata.org/pandas-docs/stable/groupby.html#groupby-dispatch):

In [73]:
grouped.agg({'C' : 'sum', 'D' : 'std'})

,C,D
A,,
bar,-1.366543,0.850083
foo,1.618672,0.916766


### Cython-optimized aggregation functions

Some common aggregations, currently only sum, mean, std, and sem, have optimized Cython implementations:

In [74]:
df.groupby('A').sum()

,C,D
A,,
bar,-1.366543,0.379664
foo,1.618672,0.561526


In [75]:
df.groupby(['A', 'B']).mean()

C         D
A   B                        
bar one   -1.281965  1.046477
    three -0.611272 -0.629967
    two    0.526694 -0.036847
foo one   -0.070764  0.611299
    three -0.475310  0.327859
    two    1.117755 -0.494465

Of course sum and mean are implemented on pandas objects, so the above code would work even without the special versions via dispatching (see below).

## Transformation

The transform method returns an object that is indexed the same (same size) as the one being grouped. Thus, the passed transform function should return a result that is the same size as the group chunk. For example, suppose we wished to standardize the data within each group: